In [1]:
new_out_path = r'\\apanzp-ainas001\Dept\ACTGRP\Infor\SysRef\Data Mart\Source_Reconciliations'

In [2]:
#Source Server and Database (main source where data is actualy fetched from)

Source_Server = 'APAUKZNUDB191.aceins.com'
Source_Database = 'KaizenProductionBackup'
Source_Schema = 'dbo'

#Destination Server and Database  (our destination database where using ETL / SSIS  data is saved at)

Destination_Server = 'APAUACTUAP161'
Destination_Database = 'DW_Actuarial'
Destination_Schema = 'KAIZEN'

In [3]:
# The table names defined in a LIST variable "table_list_src" and "table_list_dest"

table_list_SRC = ['Authority', 'Branch', 'Broker', 'Client', 'CODEX_Booking', 'CODEX_CCI', 'CODEX_Policy_Other', 'CODEX_Stamps', 
                  'CODEX_UWInfo', 'KTS_Registry', 'Nexus2_Main', 'Policy_Basic', 'Product', 'Users', 
                  'Quokka_PARTONE', 'Quokka_PARTTHREE', 'Quokka_PARTTWO']

table_list_DW = ['Authority', 'Branch', 'Broker', 'Client', 'CODEXBooking', 'CODEXCCI', 'CODEXPolicyOther', 'CODEXStamps', 
                   'CODEXUWInfo', 'KTSRegistry', 'Nexus2Main', 'PolicyBasic', 'Product', 'Users',
                 'Quokka_PARTONE', 'Quokka_PARTTHREE', 'Quokka_PARTTWO']
'''

table_list_DW = ['Authority', 'Branch', 'Broker', 'Client', 'CODEX_Booking', 'CODEX_CCI', 'CODEX_Policy_Other', 'CODEX_Stamps', 
                  'CODEX_UWInfo', 'KTS_Registry', 'Nexus2_Main', 'Policy_Basic', 'Product', 'Users', 
                  'Quokka_PARTONE', 'Quokka_PARTTHREE', 'Quokka_PARTTWO']
'''

"\n\ntable_list_DW = ['Authority', 'Branch', 'Broker', 'Client', 'CODEX_Booking', 'CODEX_CCI', 'CODEX_Policy_Other', 'CODEX_Stamps', \n                  'CODEX_UWInfo', 'KTS_Registry', 'Nexus2_Main', 'Policy_Basic', 'Product', 'Users', \n                  'Quokka_PARTONE', 'Quokka_PARTTHREE', 'Quokka_PARTTWO']\n"

In [4]:
import importlib
import subprocess

# Install packages if they are not already installed
def install_package(package_name):
    try:
        importlib.import_module(package_name)
        print(f"'{package_name}' package already exists in your system")
    except ImportError:
        subprocess.call(['pip', 'install', '--trusted-host', 'pypi.org', '--trusted-host', 'pypi.python.org', '--trusted-host', 'files.pythonhosted.org', package_name, '-q'])
        print(f"'{package_name}' package is now installed")

# Install required packages
install_package('plyer')
install_package('pyodbc')
install_package('pandas')
install_package('sqlalchemy')
install_package('sqldf')
install_package('pandasql')

'plyer' package already exists in your system
'pyodbc' package already exists in your system
'pandas' package already exists in your system
'sqlalchemy' package already exists in your system
'sqldf' package already exists in your system
'pandasql' package already exists in your system


In [5]:
import time
from plyer import notification
import pyodbc
import pandas as pd
import sqlalchemy as db
import os
from pandasql import sqldf
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.max_columns', None)
pyodbc.drivers()
import warnings
warnings.filterwarnings('ignore')

import datetime
current_date = datetime.datetime.now().strftime('%d_%m_%Y')

st_fl_time = time.time()

In [6]:


# Create SQLAlchemy engine with a connection string - DW_Actuarial - DESTINATION
conn_str = (
        "Driver={SQL Server Native Client 11.0};"
        f"Server={Source_Server};"
        f"Database={Source_Database};"
        "Trusted_Connection=yes;"
    )

engine_SRC = db.create_engine(f'mssql+pyodbc:///?odbc_connect={conn_str}')



conn_str = (
        "Driver={SQL Server Native Client 11.0};"
        f"Server={Destination_Server};"
        f"Database={Destination_Database};"
        "Trusted_Connection=yes;"
    )

engine_DW = db.create_engine(f'mssql+pyodbc:///?odbc_connect={conn_str}')

In [7]:
def KAIZEN_SRC(table_list_SRC):
    # Define queries and create a dictionary with keys as "query{i}"
    global engine_SRC
    num_names = len(table_list_SRC)
    key_names = [f"query{i}" for i in range(1, num_names+1)]
    values_names = []
    for tbl_name in table_list_SRC:
        
        query_SRC = f"""SELECT '{tbl_name}' AS TABLE_NAME_SRC,
                   COUNT(*) AS SQL_SERVER_ROW_COUNT_SRC,       
                   (SELECT COUNT(*) FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_SCHEMA = '{Source_Schema}' AND TABLE_NAME = '{tbl_name}' 
                   AND COLUMN_NAME NOT IN ('CopyDate', 'ParentExecutionID')) AS SQL_SERVER_COL_COUNT_SRC,
                   STUFF((SELECT ', ' + COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS 
                      WHERE TABLE_SCHEMA = '{Source_Schema}' AND TABLE_NAME = '{tbl_name}' AND COLUMN_NAME NOT IN ('CopyDate', 'ParentExecutionID')
                      FOR XML PATH('')), 1, 2, '') AS COLUMN_NAMES_SRC,
                   CONVERT(datetime, SWITCHOFFSET(SYSDATETIMEOFFSET(), '+10:30')) AS CEN_AUSTRALIA_STANDARD_TIME,
                   CONVERT(datetime, SWITCHOFFSET(SYSDATETIMEOFFSET(), '+05:30')) AS INDIAN_STANDARD_TIME
                   FROM {Source_Database}.{Source_Schema}.{tbl_name}"""
           
        values_names.append(query_SRC)
    queries_SRC = dict(zip(key_names, values_names))
    
    
    # Loop through the queries and execute them
    result_dict = {}
    for query_name, query_SRC in queries_SRC.items():
        # Execute query and create a Pandas DataFrame
        df = pd.read_sql(query_SRC, engine_SRC)
        result_dict[query_name] = df

    # Combine all query results into a single pandas DataFrame
    combined_df_SRC = pd.concat([result_dict[q] for q in result_dict])

    # Add a new column "query_name" to indicate which query each row belongs to
    combined_df_SRC = combined_df_SRC.assign(query_name=[q for q in result_dict for _ in range(len(result_dict[q]))])

    # Order the DataFrame by the "Table_Name" column
    combined_df_SRC = combined_df_SRC.sort_values(by=['TABLE_NAME_SRC'])

    # Return the Pandas DataFrame
    return combined_df_SRC



In [8]:
def KAIZEN_SRC_COL_DETAILS(table_list_SRC):
    # Define queries and create a dictionary with keys as "query{i}"
    global engine_SRC
    num_names = len(table_list_SRC)
    key_names = [f"query{i}" for i in range(1, num_names+1)]
    values_names = []
    for tbl_name in table_list_SRC:
        
        query_SRC = f"""SELECT TABLE_NAME AS TABLE_NAME_SRC, 
                         TABLE_SCHEMA AS TABLE_SCHEMA_SRC, 
                         COLUMN_NAME AS COLUMN_NAME_SRC,
                         CASE WHEN DATA_TYPE IN ('decimal', 'numeric') 
                              THEN CONCAT(DATA_TYPE, ' (', CHARACTER_MAXIMUM_LENGTH, NUMERIC_PRECISION, ',', NUMERIC_SCALE, ')')
                              WHEN DATA_TYPE IN ('varchar', 'char', 'nvarchar')
                              THEN CONCAT(DATA_TYPE, ' (', CHARACTER_MAXIMUM_LENGTH, ')')
                              WHEN DATA_TYPE IN ('int', 'bigint', 'datetime') THEN DATA_TYPE
                              ELSE DATA_TYPE
                              END AS DATA_TYPE_WITH_LENGTH_SRC,
                         CONVERT(datetime, SWITCHOFFSET(SYSDATETIMEOFFSET(), '+10:30')) AS CEN_AUSTRALIA_STANDARD_TIME,
                         CONVERT(datetime, SWITCHOFFSET(SYSDATETIMEOFFSET(), '+05:30')) AS INDIAN_STANDARD_TIME    
                         FROM INFORMATION_SCHEMA.COLUMNS
                         WHERE TABLE_NAME = '{tbl_name}' AND TABLE_SCHEMA = '{Source_Schema}' AND 
                               COLUMN_NAME NOT IN ('CopyDate', 'ParentExecutionID')
                         ORDER BY TABLE_NAME, COLUMN_NAME"""
           
        values_names.append(query_SRC)
    queries_SRC = dict(zip(key_names, values_names))
    
    
    # Loop through the queries and execute them
    result_dict = {}
    for query_name, query_SRC in queries_SRC.items():
        # Execute query and create a Pandas DataFrame
        df = pd.read_sql(query_SRC, engine_SRC)
        result_dict[query_name] = df

    # Combine all query results into a single pandas DataFrame
    combined_df_SRC_COL_DETAILS = pd.concat([result_dict[q] for q in result_dict])

    # Add a new column "query_name" to indicate which query each row belongs to
    combined_df_SRC_COL_DETAILS = combined_df_SRC_COL_DETAILS.assign(query_name=[q for q in result_dict for _ in range(len(result_dict[q]))])

    # Order the DataFrame by the "Table_Name" column
    combined_df_SRC_COL_DETAILS = combined_df_SRC_COL_DETAILS.sort_values(by=['TABLE_NAME_SRC', 'COLUMN_NAME_SRC'])

    # Return the Pandas DataFrame
    return combined_df_SRC_COL_DETAILS



In [9]:
def KAIZEN_DW(table_list_DW):
    # Define queries and create a dictionary with keys as "query{i}"
    global engine_DW
    num_names = len(table_list_DW)
    key_names = [f"query{i}" for i in range(1, num_names+1)]
    values_names = []
    for tbl_name in table_list_DW:
        
        query_DW = f"""SELECT '{tbl_name}' AS TABLE_NAME_DW,
                   COUNT(*) AS SQL_SERVER_ROW_COUNT_DW,       
                   (SELECT COUNT(*) FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_SCHEMA = '{Destination_Schema}' AND TABLE_NAME = '{tbl_name}' 
                   AND COLUMN_NAME NOT IN ('CopyDate', 'ParentExecutionID')) AS SQL_SERVER_COL_COUNT_DW,
                   STUFF((SELECT ', ' + COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS 
                      WHERE TABLE_SCHEMA = '{Destination_Schema}' AND TABLE_NAME = '{tbl_name}' AND COLUMN_NAME NOT IN ('CopyDate', 'ParentExecutionID')
                      FOR XML PATH('')), 1, 2, '') AS COLUMN_NAMES_DW,
                   CONVERT(datetime, SWITCHOFFSET(SYSDATETIMEOFFSET(), '+10:30')) AS CEN_AUSTRALIA_STANDARD_TIME,
                   CONVERT(datetime, SWITCHOFFSET(SYSDATETIMEOFFSET(), '+05:30')) AS INDIAN_STANDARD_TIME
                   FROM {Destination_Database}.{Destination_Schema}.{tbl_name}"""
        
        values_names.append(query_DW)
    queries_DW = dict(zip(key_names, values_names))
    
    
    # Loop through the queries and execute them
    result_dict = {}
    for query_name, query_DW in queries_DW.items():
        # Execute query and create a Pandas DataFrame
        df = pd.read_sql(query_DW, engine_DW)
        result_dict[query_name] = df

    # Combine all query results into a single pandas DataFrame
    combined_df_DW = pd.concat([result_dict[q] for q in result_dict])

    # Add a new column "query_name" to indicate which query each row belongs to
    combined_df_DW = combined_df_DW.assign(query_name=[q for q in result_dict for _ in range(len(result_dict[q]))])

    # Order the DataFrame by the "Table_Name" column
    combined_df_DW = combined_df_DW.sort_values(by=['TABLE_NAME_DW'])

    # Return the Pandas DataFrame
    return combined_df_DW

In [10]:
def KAIZEN_DW_COL_DETAILS(table_list_DW):
    # Define queries and create a dictionary with keys as "query{i}"
    global engine_DW
    num_names = len(table_list_DW)
    key_names = [f"query{i}" for i in range(1, num_names+1)]
    values_names = []
    for tbl_name in table_list_DW:
        
        query_DW = f"""SELECT TABLE_NAME AS TABLE_NAME_DW, 
                         TABLE_SCHEMA AS TABLE_SCHEMA_DW, 
                         COLUMN_NAME AS COLUMN_NAME_DW,
                         CASE WHEN DATA_TYPE IN ('decimal', 'numeric') 
                              THEN CONCAT(DATA_TYPE, ' (', CHARACTER_MAXIMUM_LENGTH, NUMERIC_PRECISION, ',', NUMERIC_SCALE, ')')
                              WHEN DATA_TYPE IN ('varchar', 'char', 'nvarchar')
                              THEN CONCAT(DATA_TYPE, ' (', CHARACTER_MAXIMUM_LENGTH, ')')
                              WHEN DATA_TYPE IN ('int', 'bigint', 'datetime') THEN DATA_TYPE
                              ELSE DATA_TYPE
                              END AS DATA_TYPE_WITH_LENGTH_DW,
                         CONVERT(datetime, SWITCHOFFSET(SYSDATETIMEOFFSET(), '+10:30')) AS CEN_AUSTRALIA_STANDARD_TIME,
                         CONVERT(datetime, SWITCHOFFSET(SYSDATETIMEOFFSET(), '+05:30')) AS INDIAN_STANDARD_TIME     
                         FROM INFORMATION_SCHEMA.COLUMNS
                         WHERE TABLE_NAME = '{tbl_name}' AND TABLE_SCHEMA = '{Destination_Schema}' AND 
                               COLUMN_NAME NOT IN ('CopyDate', 'ParentExecutionID')
                         ORDER BY TABLE_NAME, COLUMN_NAME"""
           
        values_names.append(query_DW)
    queries_DW = dict(zip(key_names, values_names))
    
    
    # Loop through the queries and execute them
    result_dict = {}
    for query_name, query_DW in queries_DW.items():
        # Execute query and create a Pandas DataFrame
        df = pd.read_sql(query_DW, engine_DW)
        result_dict[query_name] = df

    # Combine all query results into a single pandas DataFrame
    combined_df_DW_COL_DETAILS = pd.concat([result_dict[q] for q in result_dict])

    # Add a new column "query_name" to indicate which query each row belongs to
    combined_df_DW_COL_DETAILS = combined_df_DW_COL_DETAILS.assign(query_name=[q for q in result_dict for _ in range(len(result_dict[q]))])

    # Order the DataFrame by the "Table_Name" column
    combined_df_DW_COL_DETAILS = combined_df_DW_COL_DETAILS.sort_values(by=['TABLE_NAME_DW', 'COLUMN_NAME_DW'])

    # Return the Pandas DataFrame
    return combined_df_DW_COL_DETAILS



In [11]:
# Call the two functions to get table data
#combined_df_SRC = KAIZEN_SRC(table_list_SRC)
#combined_df_DW = KAIZEN_DW(table_list_DW)


#combined_df = combined_df_SRC.merge(combined_df_DW, on='query_name', how='left')

combined_df = pd.concat([KAIZEN_SRC(table_list_SRC).set_index('query_name'), KAIZEN_DW(table_list_DW).loc[:, ['query_name','TABLE_NAME_DW', 'SQL_SERVER_ROW_COUNT_DW', 'SQL_SERVER_COL_COUNT_DW', 'COLUMN_NAMES_DW']].set_index('query_name')], axis=1, join='outer')



def remove_common_words(string1, string2):
    list1 = string1.split(', ')
    list2 = string2.split(', ')
    common_words = set(list1) & set(list2)
    for word in common_words:
        list1.remove(word)
        list2.remove(word)    
    return ', '.join(list1 + list2)


# create new column with removed common words
combined_df['MISSING_COLUMN_NAMES'] = combined_df.apply(lambda x: remove_common_words(x['COLUMN_NAMES_SRC'], x['COLUMN_NAMES_DW']), axis=1)


In [12]:

# This query gves us all table details details - one below another
from pandasql import sqldf


query = """
        SELECT TABLE_NAME_SRC, TABLE_NAME_DW, SQL_SERVER_ROW_COUNT_SRC, SQL_SERVER_ROW_COUNT_DW, 
            (SQL_SERVER_ROW_COUNT_DW - SQL_SERVER_ROW_COUNT_SRC) AS `ROW_DIFFERENCE (SQLSERVERDW - SQLSERVERSRC)`,           
            SQL_SERVER_COL_COUNT_SRC, SQL_SERVER_COL_COUNT_DW, 
            (SQL_SERVER_COL_COUNT_DW - SQL_SERVER_COL_COUNT_SRC) AS `COL_DIFFERENCE (SQLSERVERDW - SQLSERVERSRC)`,
            CASE WHEN (SQL_SERVER_COL_COUNT_DW - SQL_SERVER_COL_COUNT_SRC) = 0 THEN NULL
                 ELSE MISSING_COLUMN_NAMES
            END AS MISSING_COLUMN_NAMES,
            CEN_AUSTRALIA_STANDARD_TIME,
            INDIAN_STANDARD_TIME
        FROM combined_df
"""


combined_df_KAIZEN = sqldf(query, locals())

In [13]:

# This query gives us granular column details - one below another

combined_df_SRC_COL_DETAILS = KAIZEN_SRC_COL_DETAILS(table_list_SRC)
combined_df_DW_COL_DETAILS = KAIZEN_DW_COL_DETAILS(table_list_DW)


from pandasql import sqldf


query = """
        SELECT SRC.TABLE_NAME_SRC, DW.TABLE_NAME_DW,
        SRC.COLUMN_NAME_SRC, DW.COLUMN_NAME_DW,
        SRC.DATA_TYPE_WITH_LENGTH_SRC, DW.DATA_TYPE_WITH_LENGTH_DW,
        
        CASE 
        WHEN REPLACE(SUBSTR(SRC.DATA_TYPE_WITH_LENGTH_SRC, '[[:digit:]]+'), '\\D+', '') 
             != REPLACE(SUBSTR(DW.DATA_TYPE_WITH_LENGTH_DW, '[[:digit:]]+'), '\\D+', '') 
        THEN 1
        WHEN REPLACE(REPLACE(SRC.DATA_TYPE_WITH_LENGTH_SRC, '\\d+', ''), '[^a-zA-Z]+', ', ') 
             != REPLACE(REPLACE(DW.DATA_TYPE_WITH_LENGTH_DW, '\\d+', ''), '[^a-zA-Z]+', ', ') 
        THEN 2
        WHEN REPLACE(SUBSTR(SRC.DATA_TYPE_WITH_LENGTH_SRC, '[[:digit:]]+'), '\\D+', '') 
             != REPLACE(SUBSTR(DW.DATA_TYPE_WITH_LENGTH_DW, '[[:digit:]]+'), '\\D+', '') 
             AND REPLACE(REPLACE(SRC.DATA_TYPE_WITH_LENGTH_SRC, '\\d+', ''), '[^a-zA-Z]+', ', ') 
             != REPLACE(REPLACE(DW.DATA_TYPE_WITH_LENGTH_DW, '\\d+', ''), '[^a-zA-Z]+', ', ')
        THEN 3
        ELSE 0
    END AS MISSING_DATA_CODED_INFO,
        
        SRC.CEN_AUSTRALIA_STANDARD_TIME,
        SRC.INDIAN_STANDARD_TIME
        FROM combined_df_DW_COL_DETAILS DW
        LEFT JOIN combined_df_SRC_COL_DETAILS SRC 
        ON LOWER(REPLACE(SRC.TABLE_NAME_SRC, '_', '')) = LOWER(REPLACE(DW.TABLE_NAME_DW, '_', ''))
        AND LOWER(REPLACE(SRC.COLUMN_NAME_SRC, '_', '')) = LOWER(REPLACE(DW.COLUMN_NAME_DW, '_', ''))
"""

combined_df_KAIZEN_COL_DETAILS = sqldf(query, locals())

In [14]:
path_wd=os.getcwd()
path=r'\\apanzp-ainas001\Dept\ACTGRP\Infor\SysRef\Data Mart\Source_Reconciliations'
script_name='COUNT_Python_KAIZEN.ipynb'
def writeCoverSheet( writer):

    workbook3  = writer.book
    worksheet3 = workbook3.add_worksheet('Info')
    worksheet3.set_column(1,3,60)
    yellow_sum_col = workbook3.add_format({'bold': True,'align':'center','border': 1}) # white cell font color
    yellow_sum_col.set_font_name('Times New Roman')
    yellow_sum_col.set_font_size(10)
    yellow_sum_col.set_align('center')
    prod="Production"
    if Destination_Server=='APAUACTUAP161':
        prod="UAT"
      
    worksheet3.write(1,1,'Report Generated Date ',yellow_sum_col)
    worksheet3.write(1,2,current_date,yellow_sum_col)
    worksheet3.write(2,1,'Source Server',yellow_sum_col)
    worksheet3.write(2,2,Source_Server,yellow_sum_col)
    worksheet3.write(3,1,'Source Database',yellow_sum_col)
    worksheet3.write(3,2,Source_Database,yellow_sum_col)
    worksheet3.write(4,1,'Source Schema',yellow_sum_col)
    worksheet3.write(4,2,Source_Schema,yellow_sum_col)
    worksheet3.write(5,1,'Destination Server',yellow_sum_col)
    worksheet3.write(5,2,Destination_Server,yellow_sum_col)
    worksheet3.write(6,1,'Destination Database',yellow_sum_col)
    worksheet3.write(6,2,Destination_Database,yellow_sum_col)
    worksheet3.write(7,1,'Destination Schema',yellow_sum_col)
    worksheet3.write(7,2,Destination_Schema,yellow_sum_col)

    worksheet3.write(8,1,'Environment',yellow_sum_col)
    worksheet3.write(8,2,prod,yellow_sum_col)
    worksheet3.write(9,1,'Script Path',yellow_sum_col)
    worksheet3.write(9,2,path,yellow_sum_col)
    worksheet3.write(10,1,'Script Name',yellow_sum_col)
    worksheet3.write(10,2,script_name,yellow_sum_col)
    worksheet3.write(11,1,'Working Directory of script',yellow_sum_col)
    worksheet3.write(11,2,path_wd,yellow_sum_col)

In [15]:
try:
    import xlsxwriter

    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter(new_out_path + '\\' + f"Reconciliation_{Destination_Schema}_{current_date}.xlsx", engine='xlsxwriter')

    combined_df_KAIZEN.iloc[:,0:10].style.set_properties(**{'text-align': 'center'}).to_excel(writer, sheet_name= f'{Destination_Schema}_COUNTS', startrow=0, startcol=0, header=True, index=False)
    combined_df_KAIZEN.iloc[:,5:].style.set_properties(**{'text-align': 'center'}).to_excel(writer, sheet_name= f'{Destination_Schema}_COUNTS', startrow=0, startcol=5, header=True, index=False)
    combined_df_KAIZEN_COL_DETAILS.iloc[:,0:4].style.set_properties(**{'text-align': 'center'}).to_excel(writer, sheet_name=f'{Destination_Schema}_COL_DETAILS', startrow=0, startcol=0, header=True, index=False)
    combined_df_KAIZEN_COL_DETAILS.iloc[:,4:6].style.set_properties(**{'text-align': 'center'}).to_excel(writer, sheet_name=f'{Destination_Schema}_COL_DETAILS', startrow=0, startcol=5, header=True, index=False)
    combined_df_KAIZEN_COL_DETAILS.iloc[:,6:7].style.set_properties(**{'text-align': 'center'}).to_excel(writer, sheet_name=f'{Destination_Schema}_COL_DETAILS', startrow=0, startcol=7, header=True, index=False)
    combined_df_KAIZEN_COL_DETAILS.iloc[:,7:9].style.set_properties(**{'text-align': 'center'}).to_excel(writer, sheet_name=f'{Destination_Schema}_COL_DETAILS', startrow=0, startcol=8, header=True, index=False)
    
    # Get the xlsxwriter workbook and worksheet oBNects.
    workbook1  = writer.book
    worksheet1 = writer.sheets[f'{Destination_Schema}_COUNTS']
    worksheet1.set_zoom(100)

    worksheet1.set_column('A:A', 38)
    worksheet1.set_column('B:B', 38)
    worksheet1.set_column('C:C', 29)
    worksheet1.set_column('D:D', 29)
    worksheet1.set_column('E:E', 30)
    worksheet1.set_column('F:F', 28)
    worksheet1.set_column('G:G', 28)
    worksheet1.set_column('H:H', 30)
    worksheet1.set_column('I:I', 30)
    worksheet1.set_column('J:J', 30)
    worksheet1.set_column('K:K', 30)

    border_fmt = workbook1.add_format({'bottom':2, 'top':2, 'left':2, 'right':2})
    worksheet1.conditional_format(xlsxwriter.utility.xl_range(0, 0, len(combined_df_KAIZEN.iloc[:,0:10]) , len(combined_df_KAIZEN.iloc[:,0:10].columns)  ), {'type': 'no_errors', 'format': border_fmt})

    wrap_format = workbook1.add_format({'bold': True, 'text_wrap': True, 'align':'center','border': 0})
    worksheet1.write(0, 4, "ROW_DIFFERENCE \n (SQLServerDW - SQLServerSRC)", wrap_format)
    worksheet1.write(0, 7, "COL_DIFFERENCE \n (SQLServerDW - SQLServerSRC)", wrap_format)


    yellow_sum_col = workbook1.add_format({'bg_color': '#FCF8D0','bold': True,'text_wrap': True,'align':'center','border': 0}) # white cell font color
    yellow_sum_col.set_font_name('Times New Roman')
    yellow_sum_col.set_font_size(10)
    yellow_sum_col.set_align('center')
    for i in range(2,len(combined_df_KAIZEN.iloc[:,0:10])+2):
        worksheet1.write("E{0}".format(i), '=D{0}-C{0}'.format(i), yellow_sum_col)
        worksheet1.write("H{0}".format(i), '=G{0}-F{0}'.format(i), yellow_sum_col)
        i=i+1



    # Set the format for the red color
    red_format = workbook1.add_format({'bg_color': 'red', 'bold': True})
    for i in range(2,len(combined_df_KAIZEN.iloc[:,0:10])+2):
        worksheet1.conditional_format('A{0}:A{0}'.format(i), {'type': 'formula',
                                               'criteria': '=(D{0}-C{0})>0'.format(i),
                                               'format': red_format})
        worksheet1.conditional_format('B{0}:B{0}'.format(i), {'type': 'formula',
                                               'criteria': '=(D{0}-C{0})>0'.format(i),
                                               'format': red_format})
        worksheet1.conditional_format('E{0}:E{0}'.format(i), {'type': 'formula',
                                               'criteria': '=(D{0}-C{0})>0'.format(i),
                                               'format': red_format})
        worksheet1.conditional_format('H{0}:H{0}'.format(i), {'type': 'formula',
                                               'criteria': '=(G{0}-F{0})>0'.format(i),
                                               'format': red_format})

    #-------------------------------------------------------------------
    #-------------------------------------------------------------------
    
    
    workbook2  = writer.book
    worksheet2 = writer.sheets[f'{Destination_Schema}_COL_DETAILS']
    worksheet2.set_zoom(100)

    worksheet2.set_column('A:A', 38)
    worksheet2.set_column('B:B', 38)
    worksheet2.set_column('C:C', 27)
    worksheet2.set_column('D:D', 27)
    worksheet2.set_column('E:E', 30)
    worksheet2.set_column('F:F', 30)
    worksheet2.set_column('G:G', 30)
    worksheet2.set_column('H:H', 32)
    worksheet2.set_column('I:I', 31)
    worksheet2.set_column('J:J', 31)

    border_fmt = workbook2.add_format({'bottom':2, 'top':2, 'left':2, 'right':2})
    worksheet2.conditional_format(xlsxwriter.utility.xl_range(0, 0, len(combined_df_KAIZEN_COL_DETAILS.iloc[:,0:11]) , len(combined_df_KAIZEN_COL_DETAILS.iloc[:,0:11].columns)  ), {'type': 'no_errors', 'format': border_fmt})

    wrap_format = workbook2.add_format({'bold': True, 'text_wrap': True, 'align':'center','border': 0})
    worksheet2.write(0, 4, "COLUMN_NAME_DIFFERENCE \n (COLUMN_NAME_DW - COLUMN_NAME_SRC)", wrap_format)
    
    yellow_sum_col = workbook1.add_format({'bg_color': '#FCF8D0','bold': True,'text_wrap': True,'align':'center','border': 0}) # white cell font color
    yellow_sum_col.set_font_name('Times New Roman')
    yellow_sum_col.set_font_size(10)
    yellow_sum_col.set_align('center')
    for i in range(2,len(combined_df_KAIZEN_COL_DETAILS.iloc[:,0:11])+2):
        worksheet2.write("E{0}".format(i), '=SUBSTITUTE(D{0},"_","") =SUBSTITUTE(C{0},"_","")'.format(i), yellow_sum_col)
        i=i+1
    
    
    
    
    green_format = workbook2.add_format({'bg_color': '#92D050', 'bold': True})
    yellow_format = workbook2.add_format({'bg_color': 'yellow', 'bold': True})
    purple_format = workbook2.add_format({'bg_color': '#BB89F3', 'bold': True}) 
    red_format = workbook2.add_format({'bg_color': '#DD3F3F', 'bold': True})
    
    for i in range(2,len(combined_df_KAIZEN_COL_DETAILS.iloc[:,0:10])+2):
        worksheet2.conditional_format('H{0}:H{0}'.format(i), {'type': 'formula',
                                                   'criteria': '=H{0}=0'.format(i),
                                                   'format': green_format})
        worksheet2.conditional_format('H{0}:H{0}'.format(i), {'type': 'formula',
                                                   'criteria': '=H{0}=1'.format(i),
                                                   'format': yellow_format})
        worksheet2.conditional_format('H{0}:H{0}'.format(i), {'type': 'formula',
                                                   'criteria': '=H{0}=2'.format(i),
                                                   'format': purple_format})
        worksheet2.conditional_format('H{0}:H{0}'.format(i), {'type': 'formula',
                                                   'criteria': '=H{0}=3'.format(i),
                                                   'format': red_format})
    writeCoverSheet(writer)
    writer.close()
    writer.handles = None
except PermissionError:
    notification.notify(
    title = f"PERMISSION ERROR",
    message = "Your file is EITHER Open or Being currently used. Kindly close it and rerun the code",
    timeout = 10
    )

In [16]:
end_fl_time = time.time()
hours, rem = divmod(end_fl_time-st_fl_time, 3600)
minutes, seconds = divmod(rem, 60)


print("Total time taken to load the file:")
print("-----------------------------------")
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

notification.notify(
title = f"The {Destination_Schema} COUNT file",
message = "The Excel file is now generated. TIME TAKEN {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds),
timeout = 15
)

Total time taken to load the file:
-----------------------------------
00:01:23.05
